# DeBERTa-v3 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from mlflow.sklearn import save_model

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

import sentencepiece
import os

from torch.utils.data import Dataset, DataLoader
import torch

from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoModel, AutoTokenizer, AutoConfig

Install requirements in terminal:
pip install transformers torch datasets

In [6]:
df = pd.read_csv('../data/data_small.csv')

Y = df["logical_fallacies"]
X = df["text"]

In [7]:
df['text'].isna().sum()

0

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         5000 non-null   int64 
 1   dataset            5000 non-null   int64 
 2   text               5000 non-null   object
 3   logical_fallacies  5000 non-null   object
 4   source             240 non-null    object
dtypes: int64(2), object(3)
memory usage: 195.4+ KB


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.30, random_state=42)

### Data Preparation

https://stackoverflow.com/questions/76868251/how-to-load-deberta-v3-properly

In [10]:
# tokenization after train test split to prevent data leakage

#added use_fast=False to prevent tokenization error (might happen when using fast tokenization)
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base', use_fast=False)

In [11]:
def tokenize(texts):
    return tokenizer(
        texts,
        padding="max_length", #ensures that all tokenized sequences are padded to the same length, padding adds special tokens to shorter sequeces so they match the maximum length
        truncation=True, #if sequence exceeds max, it will be trucated
        max_length=512, #for most transformer models, 512 is a common limit for maximum length
        return_tensors="pt" #converts the output to pytorch tensors
    )

In [12]:
train_encodings = tokenize(X_train.to_list())
test_encodings = tokenize(X_test.to_list())

### Convert string labels to integers

In [13]:
le = LabelEncoder()
y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)

### Dataset Preparation for usage in model

needed to create a PyTorch Dataset object that:
- organizes tokenized text
- pairs them with corresponding labels
- structures everything for batch processing during training

In [14]:
#object oriented programming (class is the object), with class you can do different things, such as calling functions

class TextDataset(Dataset):  # Inherits from PyTorch's Dataset class
    def __init__(self, encodings, labels):
        self.input_ids = encodings['input_ids']       # Token IDs from tokenizer
        self.attention_mask = encodings['attention_mask']  # Mask for padding
        self.labels = torch.tensor(labels)  # Convert labels to tensors
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],       # Token IDs for one sample
            'attention_mask': self.attention_mask[idx],  # Mask for one sample
            'labels': self.labels[idx]              # Label for one sample
        }
    def __len__(self):
        return len(self.labels)  # Total number of samples

In [15]:
train_dataset = TextDataset(train_encodings, y_train)
test_dataset = TextDataset(test_encodings, y_test)

### Zero Shot Inference 

In [95]:
# disable upper limit for memory
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

# Allows up to 100% of available memory
torch.mps.set_per_process_memory_fraction(1.0)  

torch.mps.empty_cache()  # Clears unused GPU memory

In [96]:
# Load fresh copy of base model (not train on our data)
num_classes = len(df["logical_fallacies"].unique())
base_model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-small",
    num_labels=num_classes,
    problem_type="single_label_classification"
)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
def predict(model, encodings, batch_size=8):
    # Set the model to evaluation mode
    model.eval()
    
    # Use GPU
    device = torch.device("mps")
    model.to(device)
    
    # Perform inference
    probabilities = []
    for i in range(0, len(encodings["input_ids"]), batch_size):
        with torch.no_grad():
            batch = {
                "input_ids": encodings["input_ids"][i:i+batch_size].to(device),
                "attention_mask": encodings["attention_mask"][i:i+batch_size].to(device)
            }
            outputs = model(**batch)
            probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
            probabilities.extend(probs)
            
        # Clear GPU memory after each batch
        torch.mps.empty_cache()
    
    return np.array(probabilities)

In [98]:
# Get predictions for test data
base_probs = predict(base_model, test_encodings, batch_size=8)

In [99]:
# Get highest probability indices
predicted_indices = np.argmax(base_probs, axis=1)  

In [100]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_test, predicted_indices, target_names=le.classes_)
print(report)

                       precision    recall  f1-score   support

           ad_hominem       0.00      0.00      0.00       159
  appeal_to_authority       0.00      0.00      0.00        97
    appeal_to_emotion       0.00      0.00      0.00       227
        false_dilemma       0.09      1.00      0.16       133
faulty_generalization       0.00      0.00      0.00       202
                 none       0.00      0.00      0.00       682

             accuracy                           0.09      1500
            macro avg       0.01      0.17      0.03      1500
         weighted avg       0.01      0.09      0.01      1500



/Users/maren/Library/CloudStorage/OneDrive-Persönlich/Dokumente/neue_fische/Capstone Project/Capstone_project/backend/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maren/Library/CloudStorage/OneDrive-Persönlich/Dokumente/neue_fische/Capstone Project/Capstone_project/backend/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maren/Library/CloudStorage/OneDrive-Persönlich/Dokumente/neue_fische/Capstone Project/Capstone_project/backend/.venv/lib/python3.11/site-pac

Note: This deberta model is actually not designed for zero shot, there is one by MoritzLauer which can be used without requiring training on data. So training on data is actually necessary! The DeBERTa used here is meant for supervised learning. 
Another option is to use BART, facebook/bart-large-mnli model.

**Zero-Shot Learning** </span> is a concept, that a model when trained on enough unlabeled data (unsupervised learning) is able to generalize/ recognize at inference time even though the model was not trained on the inference data. This can be used in NLP, Images etc.

### Model Initialization

I had to change configuration of accelerate, as it might still be configured to fp16 (mixed precision):
- type in bash accelerate confic
- this machine
- no distributed training
- do you want to run your training on CPU only, say No, as MAC has GPU
- do you wish to optimize script with torch dynamo: say "No" if using an Apple M1 Pro with MPS backend
- do you want to use mixed prexision: NO

In [36]:
num_classes = len(df["logical_fallacies"].unique())
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-small",
    num_labels=num_classes,
    problem_type="single_label_classification"
)

model.gradient_checkpointing_enable()  # force model to use gradient checkpointing to save memory

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Class imbalance (We could try)

In [ ]:
# # Class-balanced trainer
# from sklearn.utils.class_weight import compute_class_weight
# class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

### Training Confguration 

In [31]:
training_args = TrainingArguments(
    # output_dir='./results', # to sve results
    num_train_epochs=3,
    per_device_train_batch_size=4, #small to save memory
    per_device_eval_batch_size=8, #small to save memory
    learning_rate=2e-5, #standard for deberta; maybe try 6e-6
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True
)

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {'accuracy': accuracy_score(p.label_ids, preds)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

## Execute Training

In [127]:
torch.mps.empty_cache()  # Clears unused GPU memory

In [18]:
# disable upper limit for memory
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

# Allows up to 100% of available memory
torch.mps.set_per_process_memory_fraction(1.0)  

In [30]:
# trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.939800,0.961467,0.682667
2,0.740400,0.926966,0.731333
3,0.550900,1.017781,0.749333


Test Accuracy: 0.7313


In [ ]:
results = trainer.evaluate()
print(f"Test Accuracy: {results['eval_accuracy']:.4f}")

In [37]:
output = trainer.predict(test_dataset)
predictions = np.argmax(output.predictions, axis=1)
y_true = output.label_ids

In [38]:
# Generate classification report
print(classification_report(y_true, predictions, target_names=le.classes_))

# Generate confusion matrix
cm = confusion_matrix(y_true, predictions)
print("Confusion Matrix:")
print(cm)

                       precision    recall  f1-score   support

           ad_hominem       0.71      0.60      0.65       159
  appeal_to_authority       0.69      0.32      0.44        97
    appeal_to_emotion       0.73      0.66      0.69       227
        false_dilemma       0.82      0.56      0.67       133
faulty_generalization       0.58      0.63      0.60       202
                 none       0.77      0.91      0.83       682

             accuracy                           0.73      1500
            macro avg       0.72      0.61      0.65      1500
         weighted avg       0.73      0.73      0.72      1500

Confusion Matrix:
[[ 96   6  25   2  11  19]
 [  6  31   9   0  14  37]
 [ 11   2 150   5  24  35]
 [  3   0   5  75   9  41]
 [  6   3  12   2 127  52]
 [ 14   3   5   7  35 618]]


### Save model

I saved the model here as a SK model, although it is a Pytorch model. Keep that in mind!

Save model:
import mlflow.pytorch
mlflow.pytorch.save_model(model, "deberta_model")

Load model (correct way):
model = mlflow.pytorch.load_model("deberta_model")

In [17]:
path = "../models/LLM_deberta_v3_small"

In [ ]:
save_model(sk_model=model, path=path)

### Load model

In [29]:
import mlflow.sklearn
model = mlflow.sklearn.load_model(path)

### Make predictions based on reloaded model

Here the code works, although I loaded it as a Sklearn model, because I manually converted the logict to probabilites with torch.softmax. 

mlflow.sklearn.load_model() accidentally worked because MLflow can sometimes load PyTorch models as generic Python objects, but this isn't reliable

In [ ]:
# # batch can be changed both codes, now used the upper one
# # upper one makes it more generalized for dynamic inputs, as the lower one only handles input_ids and attention_mask
# batch = {
#     key: val[i:i+batch_size].to(device) 
#     for key, val in encodings.items()
# }

# batch = {
#                 "input_ids": encodings["input_ids"][i:i+batch_size].to(device),
#                 "attention_mask": encodings["attention_mask"][i:i+batch_size].to(device)
#             }

In [30]:
## Function for prediction

def predict(model, encodings, batch_size=8):
    # Set the model to evaluation mode
    model.eval()
    
    # Use GPU
    device = torch.device("mps")
    model.to(device)
    
    # Perform inference
    probabilities = []
    for i in range(0, len(encodings["input_ids"]), batch_size):
        with torch.no_grad():
            batch = {
                key: val[i:i+batch_size].to(device) 
                for key, val in encodings.items()
            }
            outputs = model(**batch)
            probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
            probabilities.extend(probs)
            
        # Clear GPU memory after each batch
        torch.mps.empty_cache()
    
    return np.array(probabilities)

In [31]:
#needed to reduce the batch size, otherwise I had an error
# Get predictions for test data
base_probs = predict(model, test_encodings, batch_size=2)

In [32]:
# Get highest probability indices
predicted_labels = np.argmax(base_probs, axis=1)

In [33]:
#probabilites
predicted_label_probs = base_probs[np.arange(len(predicted_labels)), predicted_labels]

In [34]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_test, predicted_labels, target_names=le.classes_)
print(report)

                       precision    recall  f1-score   support

           ad_hominem       0.71      0.60      0.65       159
  appeal_to_authority       0.69      0.32      0.44        97
    appeal_to_emotion       0.73      0.66      0.69       227
        false_dilemma       0.82      0.56      0.67       133
faulty_generalization       0.58      0.63      0.60       202
                 none       0.77      0.91      0.83       682

             accuracy                           0.73      1500
            macro avg       0.72      0.61      0.65      1500
         weighted avg       0.73      0.73      0.72      1500

